In [63]:
import pandas as pd
from grid_search_utils import get_grid_search_results
import itertools

In [178]:
# give directory name
directory = "07-03"
get_grid_search_results(directory)

     attributes  values  game_size  vocab_size_factor  n_epochs  batch_size  \
0           3.0     4.0       10.0                3.0      60.0        32.0   
1           3.0     4.0       10.0                3.0      60.0        32.0   
2           3.0     4.0       10.0                3.0      60.0        32.0   
3           3.0    16.0       10.0                3.0      60.0        64.0   
4           3.0    16.0       10.0                3.0      60.0        64.0   
..          ...     ...        ...                ...       ...         ...   
283         3.0    16.0       10.0                3.0      60.0        32.0   
284         3.0    16.0       10.0                3.0      60.0        64.0   
285         3.0    16.0       10.0                3.0      60.0        64.0   
286         3.0    16.0       10.0                3.0      60.0        64.0   
287         3.0    16.0       10.0                3.0      60.0        64.0   

     learning_rate  hidden_size  temp_update  tempe

In [179]:
data = pd.read_csv("results_" + directory + ".csv")

## New grid search results

In [180]:
# parameters that were tested
for col in data.columns[:-4]:
    print(col, data[col].unique())    

attributes [3 5]
values [ 4 16]
game_size [10]
vocab_size_factor [3.]
n_epochs [60.]
batch_size [ 32  64 128]
learning_rate [0.0005 0.001 ]
hidden_size [128 256]
temp_update [0.97 0.99]
temperature [1.  1.5 2. ]


In [181]:
data = data[((data['attributes'] == 3) & (data['values'] == 4))
            | ((data['attributes'] == 3) & (data['values'] == 16))
            | ((data['attributes'] == 5) & (data['values'] == 4))]

In [182]:
attributes = [3, 5]
values = [4, 16]
datasets = list(itertools.product(attributes, values))[:3]
index = data.index
for (attributes, values) in datasets:
    print("dataset:", attributes, values)
    per_dataset = data[(data["attributes"] == attributes) & (data["values"] == values)]
    max_test_acc = per_dataset['test_accuracy'].idxmax()
    print(per_dataset.loc[index[max_test_acc]])

dataset: 3 4
attributes             3.000000
values                 4.000000
game_size             10.000000
vocab_size_factor      3.000000
n_epochs              60.000000
batch_size            32.000000
learning_rate          0.001000
hidden_size          128.000000
temp_update            0.990000
temperature            1.500000
train_loss             0.018696
train_accuracy         0.995069
test_loss              0.025826
test_accuracy          0.990799
Name: 15, dtype: float64
dataset: 3 16
attributes             3.000000
values                16.000000
game_size             10.000000
vocab_size_factor      3.000000
n_epochs              60.000000
batch_size            32.000000
learning_rate          0.001000
hidden_size          256.000000
temp_update            0.990000
temperature            1.500000
train_loss             0.012811
train_accuracy         0.994812
test_loss              0.035555
test_accuracy          0.990956
Name: 93, dtype: float64
dataset: 5 4
attributes    

### Batch size
Highest mean test accuracies with batch size 64, highest maximum test accuracies with batch size 32.

In [183]:
bs_sumstat = data.groupby(['attributes', 'values', 'batch_size'])['train_accuracy', 'test_accuracy'].describe()
bs_train = bs_sumstat.loc[:,[('train_accuracy', 'max'), ('train_accuracy', 'mean'), ('train_accuracy', 'std')]]
bs_test = bs_sumstat.loc[:,[('test_accuracy', 'max'), ('test_accuracy', 'mean'), ('test_accuracy', 'std')]]
bs_merged = bs_train.merge(bs_test, left_index=True, right_index=True)
bs_merged

c:\Users\krkobrock\miniconda3\envs\egg\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


train_accuracy                     test_accuracy  \
                                        max      mean       std           max   
attributes values batch_size                                                    
3          4      32               0.996144  0.943500  0.089889      0.990799   
                  64               0.992631  0.954048  0.027507      0.980894   
                  128              0.998038  0.922988  0.085802      0.978669   
           16     32               0.998686  0.925043  0.123395      0.990956   
                  64               0.986833  0.956103  0.025236      0.975180   
                  128              0.987758  0.921175  0.083495      0.974911   
5          4      32               0.997766  0.934680  0.119090      0.990893   
                  64               0.997459  0.941903  0.085098      0.996113   
                  128              0.983765  0.926506  0.080721      0.964336   

                                                  
                                  mean       std  
attributes values batch_size                      
3          4      32          0.933245  0.087554  
                  64          0.936850  0.025317  
                  128         0.910857  0.085462  
           16     32          0.912311  0.124454  
                  64          0.944313  0.025264  
                  128         0.909678  0.081154  
5          4      32          0.921168  0.121215  
                  64          0.928727  0.084403  
                  128         0.913966  0.078052

### Learning rate
Higher maximum accuracies with learning rate 0.001, but higher mean accuracies with learning rate 0.0005.

In [184]:
lr_sumstat = data.groupby(['attributes', 'values', 'learning_rate'])['train_accuracy', 'test_accuracy'].describe()
lr_train = lr_sumstat.loc[:,[('train_accuracy', 'max'), ('train_accuracy', 'mean'), ('train_accuracy', 'std')]]
lr_test = lr_sumstat.loc[:,[('test_accuracy', 'max'), ('test_accuracy', 'mean'), ('test_accuracy', 'std')]]
lr_merged = lr_train.merge(lr_test, left_index=True, right_index=True)
lr_merged

c:\Users\krkobrock\miniconda3\envs\egg\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


train_accuracy                      \
                                           max      mean       std   
attributes values learning_rate                                      
3          4      0.0005              0.996144  0.949047  0.028927   
                  0.0010              0.998038  0.931310  0.099942   
           16     0.0005              0.991247  0.949208  0.022800   
                  0.0010              0.998686  0.919006  0.120497   
5          4      0.0005              0.995376  0.953913  0.023992   
                  0.0010              0.997766  0.914813  0.130691   

                                test_accuracy                      
                                          max      mean       std  
attributes values learning_rate                                    
3          4      0.0005             0.986477  0.935239  0.028492  
                  0.0010             0.990799  0.918729  0.097868  
           16     0.0005             0.980484  0.937910  0.022415  
                  0.0010             0.990956  0.906291  0.120079  
5          4      0.0005             0.986942  0.940678  0.025455  
                  0.0010             0.996113  0.901897  0.130359

### Hidden size
Highest mean accuracies with hidden size 128.

In [185]:
hs_sumstat = data.groupby(['attributes', 'values', 'hidden_size'])['train_accuracy', 'test_accuracy'].describe()
hs_train = hs_sumstat.loc[:,[('train_accuracy', 'max'), ('train_accuracy', 'mean'), ('train_accuracy', 'std')]]
hs_test = hs_sumstat.loc[:,[('test_accuracy', 'max'), ('test_accuracy', 'mean'), ('test_accuracy', 'std')]]
hs_merged = hs_train.merge(hs_test, left_index=True, right_index=True)
hs_merged

c:\Users\krkobrock\miniconda3\envs\egg\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


train_accuracy                      \
                                         max      mean       std   
attributes values hidden_size                                      
3          4      128               0.995069  0.944307  0.027819   
                  256               0.998038  0.936050  0.100886   
           16     128               0.994435  0.946345  0.027500   
                  256               0.998686  0.921869  0.120184   
5          4      128               0.996076  0.949463  0.021104   
                  256               0.997766  0.919264  0.132391   

                              test_accuracy                      
                                        max      mean       std  
attributes values hidden_size                                    
3          4      128              0.990799  0.930737  0.028195  
                  256              0.986477  0.923231  0.098520  
           16     128              0.984124  0.934907  0.025772  
                  256              0.990956  0.909294  0.120142  
5          4      128              0.973438  0.936534  0.018819  
                  256              0.996113  0.906041  0.132600

### Temperature
Highest mean accuracies with temperature 2.

In [186]:
t_sumstat = data.groupby(['attributes', 'values', 'temperature'])['train_accuracy', 'test_accuracy'].describe()
t_train = t_sumstat.loc[:,[('train_accuracy', 'max'), ('train_accuracy', 'mean'), ('train_accuracy', 'std')]]
t_test = t_sumstat.loc[:,[('test_accuracy', 'max'), ('test_accuracy', 'mean'), ('test_accuracy', 'std')]]
t_merged = t_train.merge(t_test, left_index=True, right_index=True)
t_merged

c:\Users\krkobrock\miniconda3\envs\egg\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


train_accuracy                      \
                                         max      mean       std   
attributes values temperature                                      
3          4      1.0               0.980742  0.899930  0.112644   
                  1.5               0.996144  0.958680  0.024980   
                  2.0               0.998038  0.961925  0.029422   
           16     1.0               0.991247  0.902898  0.116464   
                  1.5               0.994812  0.935416  0.087120   
                  2.0               0.998686  0.964007  0.022250   
5          4      1.0               0.982390  0.876628  0.147288   
                  1.5               0.997459  0.959905  0.023857   
                  2.0               0.997766  0.966556  0.023673   

                              test_accuracy                      
                                        max      mean       std  
attributes values temperature                                    
3          4      1.0              0.985587  0.890990  0.111896  
                  1.5              0.990799  0.944676  0.026058  
                  2.0              0.986477  0.945285  0.028221  
           16     1.0              0.980484  0.892970  0.118115  
                  1.5              0.990956  0.923078  0.085194  
                  2.0              0.989078  0.950253  0.023796  
5          4      1.0              0.984969  0.865971  0.148495  
                  1.5              0.996113  0.946634  0.025948  
                  2.0              0.990893  0.951257  0.022317

### Temperature update
Highest mean accuracies for temperature update 0.99.

In [187]:
tu_sumstat = data.groupby(['attributes', 'values', 'temp_update'])['train_accuracy', 'test_accuracy'].describe()
tu_train = tu_sumstat.loc[:,[('train_accuracy', 'max'), ('train_accuracy', 'mean'), ('train_accuracy', 'std')]]
tu_test = tu_sumstat.loc[:,[('test_accuracy', 'max'), ('test_accuracy', 'mean'), ('test_accuracy', 'std')]]
tu_merged = tu_train.merge(tu_test, left_index=True, right_index=True)
tu_merged

c:\Users\krkobrock\miniconda3\envs\egg\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


train_accuracy                      \
                                         max      mean       std   
attributes values temp_update                                      
3          4      0.97              0.986841  0.931699  0.073344   
                  0.99              0.998038  0.948658  0.073891   
           16     0.97              0.983124  0.927662  0.073676   
                  0.99              0.998686  0.940552  0.099974   
5          4      0.97              0.987410  0.921098  0.112484   
                  0.99              0.997766  0.947628  0.073670   

                              test_accuracy                      
                                        max      mean       std  
attributes values temp_update                                    
3          4      0.97             0.980894  0.920317  0.070706  
                  0.99             0.990799  0.933651  0.073752  
           16     0.97             0.975086  0.917051  0.075995  
                  0.99             0.990956  0.927150  0.098020  
5          4      0.97             0.979123  0.907871  0.114634  
                  0.99             0.996113  0.934704  0.070027